<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/KB%20crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from  google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 크롤링(Selenium, BeautifulSoup)
- Selenium으로 webdriver를 생성해 동적 크롤링 수행 후,
- 가져온 html 데이터를 BeatifulSoup으로 파싱

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [833 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [977 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,231 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,099 kB]
Get:13 http://archive.ubuntu.co

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd
import random

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# driver=webdriver.Chrome()

In [ ]:
from selenium.webdriver.common.by import By
import re


# 페이지 접근
url = f"https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
driver.get(url)
driver.implicitly_wait(random.randint(3,7))

# 카테고리 정보 가져오기
# (이후에 카테고리별 법령 개수랑 크롤링된 개수랑 비교하면서 반복문 탈출필요)
# for i in range(2,47): #시작이 2부터임
for i in range(42,47):
  cat_button = driver.find_element(By.XPATH, f'//*[@id="tree_{i}_a"]')
  cat_name = cat_button.text.split()[1]
  cat_cnt = int(cat_button.text.split('(')[-1].replace(')',''))
  print(f'\n>>> {cat_name} 카테고리 크롤링 시작...')

  # 카테고리 클릭
  driver.execute_script("arguments[0].click();", cat_button)
  driver.implicitly_wait(random.randint(3,7))

  # 페이지 클릭
  final_df = pd.DataFrame()
  tot_cnt = 0
  for p in range(2,10):
    # 페이지 내 특정 html 요소로 접근
    driver.switch_to.frame('indexFrame')
    if p > 2:
      page_button = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[3]/a[{p}]')
      page_button.click()
      driver.implicitly_wait(random.randint(3,7))

    # 페이지 내 각 법령마다 처리해주기: 20개씩 나와있음
    for i in range(1, 21):
      tot_cnt += 1

      # 해당 카테고리의 법령 개수보다 크롤링된 법령 개수가 많아지면 반복문 탈출
      if tot_cnt > cat_cnt:
        break

      # 세부 페이지로 접근
      try:
        driver.switch_to.frame('indexFrame')
      except:
        pass
      subpage_button = driver.find_element(By.XPATH, f'//*[@id="{str(i)}"]/td[2]/p/a')
      subpage_name = subpage_button.text.split('\n')[0]

      driver.execute_script("arguments[0].click();", subpage_button)
      driver.implicitly_wait(random.randint(3,7))

      # 영문/한글 체크 선택
      driver.switch_to.window(driver.window_handles[-1])
      driver.get(driver.current_url.replace("ENG","KOR"))
      driver.implicitly_wait(random.randint(3,7))

      # 페이지 내 특정 html 요소로 접근
      driver.switch_to.frame('lawViewContent')
      driver.implicitly_wait(random.randint(3,7))

      # 행, 호, 글 부분만 선택해주기
      r = driver.page_source
      soup = BeautifulSoup(r, "html.parser")

      # 한글, 영어 매칭해서 df로 저장
      df = pd.DataFrame()
      for c in ['none', 'hang']:
        document = soup.find_all('div', class_=c)
        kor, eng = [], []
        kor_cnt, eng_cnt = 0, 0
        for d in document:
          d_text = d.get_text()
          if not d_text or (d_text[0] == '[' and d_text[-1] == ']'):
            continue
          else:
            pattern = re.compile('[가-힣]+')
            if len(pattern.findall(d_text)) >= 1:   # 한글 부분
              # if eng_cnt == 1:
                kor.append(d_text)
                kor_cnt += 1
            else:              # 영어 부분
              if kor_cnt != 0 and eng_cnt != 0:
                if kor_cnt == eng_cnt:
                  df = pd.concat([df, pd.DataFrame({'category':cat_name, 'law_name':subpage_name, 'en':eng, 'ko':kor})],
                                 ignore_index=True)
                kor, eng = [], []
                kor_cnt, eng_cnt = 0, 0
              eng.append(d_text)
              eng_cnt += 1

      # 각 법령 문서 취합
      print(f'{tot_cnt}번째 법령 취합 완료: row 총 {len(df)}개')
      final_df = pd.concat([final_df, df], ignore_index=True)

      # 다시 원래 위치로 되돌아가기
      driver.close()
      driver.switch_to.window(driver.window_handles[0])
      driver.implicitly_wait(random.randint(3,7))

    # 페이지 수준 반복문 탈출(해당 카테고리 법령 수집 완료)
    if tot_cnt > cat_cnt:
        break

  # 해당 카테고리 데이터 저장
  print(f'>>> {cat_name} 카테고리 크롤링 완료...(row 총 {len(final_df)}개)\n')
  final_df.to_csv(f'/content/drive/MyDrive/KBdata/{cat_name}_df.csv')


>>> 육운·항공·관광 카테고리 크롤링 시작...
1번째 법령 취합 완료: row 총 122개
2번째 법령 취합 완료: row 총 85개
3번째 법령 취합 완료: row 총 57개
4번째 법령 취합 완료: row 총 260개
5번째 법령 취합 완료: row 총 16개
6번째 법령 취합 완료: row 총 33개
7번째 법령 취합 완료: row 총 357개
8번째 법령 취합 완료: row 총 217개
9번째 법령 취합 완료: row 총 196개
10번째 법령 취합 완료: row 총 146개
11번째 법령 취합 완료: row 총 378개
12번째 법령 취합 완료: row 총 50개
13번째 법령 취합 완료: row 총 86개
14번째 법령 취합 완료: row 총 107개
15번째 법령 취합 완료: row 총 216개
16번째 법령 취합 완료: row 총 134개
17번째 법령 취합 완료: row 총 66개
18번째 법령 취합 완료: row 총 265개
19번째 법령 취합 완료: row 총 265개
20번째 법령 취합 완료: row 총 158개
21번째 법령 취합 완료: row 총 424개
22번째 법령 취합 완료: row 총 108개
23번째 법령 취합 완료: row 총 542개
24번째 법령 취합 완료: row 총 176개
25번째 법령 취합 완료: row 총 229개
26번째 법령 취합 완료: row 총 101개
27번째 법령 취합 완료: row 총 145개
28번째 법령 취합 완료: row 총 37개
29번째 법령 취합 완료: row 총 139개
30번째 법령 취합 완료: row 총 127개
31번째 법령 취합 완료: row 총 373개
32번째 법령 취합 완료: row 총 191개
33번째 법령 취합 완료: row 총 73개
34번째 법령 취합 완료: row 총 65개
35번째 법령 취합 완료: row 총 49개
36번째 법령 취합 완료: row 총 77개
37번째 법령 취합 완료: row 총 24개
38번째 법령 취합 완료: row 총 195개
39번째 

In [ ]:
final_df

,category,law_name,en,ko
0,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,The purpose of this Act is to provide for matt...,이 법은 「대한민국과 아메리카합중국 간의 상호방위조약 제4조에 의한 시설과 구역 및...
1,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,The term “Korean national worker(s) for the Un...,"이 법에서 ""주한미군 소속 한국인 근로자""란 다음 각 호의 어느 하나에 해당하는 사..."
2,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,A Korean national worker for the United States...,주한미군 소속 한국인 근로자는 이 법에 따른 지원금을 지급받으려는 때에는 대통령령으...
3,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(1) Where Korean national workers for the Unit...,① 국가는 기존 방위비분담특별협정이 종료되고 다음 방위비분담특별협정이 발효되지 아니...
4,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(2) A subsidy paid under paragraph (1) shall b...,② 제1항에 따라 지급되는 지원금은 「고용보험법」 제46조제1항제1호에 따른 금액으...
5,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(3) The detailed method for calculation and pa...,"③ 제1항 및 제2항에 따른 지원금의 구체적인 산정, 지급 방법 및 지급 기간 등에..."
6,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(1) Upon the making of a decision to or not to...,① 국방부장관이 지원금을 지급하거나 지급하지 아니하기로 결정한 때에는 7일 이내에 ...
7,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(2) The provisions of the Civil Procedure Act ...,② 제1항의 송달에 관하여는 「민사소송법」의 송달에 관한 규정을 준용한다.
8,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(1) A Korean national worker for the United St...,① 국방부장관이 제5조에 따라 결정한 사항에 대하여 이의가 있는 주한미군 소속 한국...
9,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"(2) The Minister of National Defense shall, wi...",② 국방부장관은 이의신청을 받은 날부터 7일 이내에 그 이의신청에 대하여 결정하고 ...


# 2. 데이터 결합
- 크롤링 과정에서 분리저장된 조각파일들을 결합

In [ ]:
# 크롤링 파일조각들 df형태로 리스트화

import pandas as pd
import glob


allFile_list = glob.glob('/content/drive/MyDrive/KBdata/*_df.csv')

allData = []
for file in allFile_list:
    df = pd.read_csv(file)
    allData.append(df.iloc[:,1:])
len(allData)

45


In [ ]:
# 각 df들 전부 결합(merge)

dataMerge = pd.concat(allData, axis=0, ignore_index=True)
dataMerge

,category,law_name,en,ko
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발ㆍ운영의 지원 및 개성공업지구에 투자하거나 출입ㆍ체류하는...
1,헌법,개성공업지구 지원에 관한 법률,The terms used in this Act are defined as foll...,"이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2009. 1. 30., 2..."
2,헌법,개성공업지구 지원에 관한 법률,This Act shall prevail over other statutes wit...,"개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적..."
3,헌법,개성공업지구 지원에 관한 법률,The Government may grant or lend to a local en...,정부는 개성공업지구 현지기업에 「남북협력기금법」 에 따른 남북협력기금을 지원 또는 ...
4,헌법,개성공업지구 지원에 관한 법률,In addition to the business support under this...,이 법에 정한 기업 지원에 관한 사항 이외에 다른 법률이 정하고 있는 인력ㆍ기술개발...
...,...,...,...,...
279015,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(1) A Korean national worker for the United St...,① 국방부장관이 제5조에 따라 결정한 사항에 대하여 이의가 있는 주한미군 소속 한국...
279016,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"(2) The Minister of National Defense shall, wi...",② 국방부장관은 이의신청을 받은 날부터 7일 이내에 그 이의신청에 대하여 결정하고 ...
279017,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(3) Procedures for appeal and giving notice of...,③ 제1항과 제2항에 따른 신청 절차와 결정 통지 등에 필요한 사항은 국방부령으로 ...
279018,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"(1) Costs necessary to pay subsidies, etc. und...",① 이 법에 따른 지원금의 지급 등에 필요한 비용은 국가에서 부담한다.


In [ ]:
dataMerge.duplicated().sum()

2662

In [ ]:
# 중복행 제거

dataMerge = dataMerge.drop_duplicates()
print(len(dataMerge))

276358


In [ ]:
dataMerge

,category,law_name,en,ko
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발ㆍ운영의 지원 및 개성공업지구에 투자하거나 출입ㆍ체류하는...
1,헌법,개성공업지구 지원에 관한 법률,The terms used in this Act are defined as foll...,"이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2009. 1. 30., 2..."
2,헌법,개성공업지구 지원에 관한 법률,This Act shall prevail over other statutes wit...,"개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적..."
3,헌법,개성공업지구 지원에 관한 법률,The Government may grant or lend to a local en...,정부는 개성공업지구 현지기업에 「남북협력기금법」 에 따른 남북협력기금을 지원 또는 ...
4,헌법,개성공업지구 지원에 관한 법률,In addition to the business support under this...,이 법에 정한 기업 지원에 관한 사항 이외에 다른 법률이 정하고 있는 인력ㆍ기술개발...
...,...,...,...,...
279015,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(1) A Korean national worker for the United St...,① 국방부장관이 제5조에 따라 결정한 사항에 대하여 이의가 있는 주한미군 소속 한국...
279016,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"(2) The Minister of National Defense shall, wi...",② 국방부장관은 이의신청을 받은 날부터 7일 이내에 그 이의신청에 대하여 결정하고 ...
279017,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(3) Procedures for appeal and giving notice of...,③ 제1항과 제2항에 따른 신청 절차와 결정 통지 등에 필요한 사항은 국방부령으로 ...
279018,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"(1) Costs necessary to pay subsidies, etc. und...",① 이 법에 따른 지원금의 지급 등에 필요한 비용은 국가에서 부담한다.


In [ ]:
# 최종 파일로 저장

dataMerge.to_csv('/content/drive/MyDrive/KBdata/final_df.csv', index=False)

# 3. 전처리
- 필요없는 불용어들 제거: (,),[,],<,>,:,; 등

In [419]:
# 파일 불러오기

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/KBdata/final_df.csv')
df

,category,law_name,en,ko
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발ㆍ운영의 지원 및 개성공업지구에 투자하거나 출입ㆍ체류하는...
1,헌법,개성공업지구 지원에 관한 법률,The terms used in this Act are defined as foll...,"이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2009. 1. 30., 2..."
2,헌법,개성공업지구 지원에 관한 법률,This Act shall prevail over other statutes wit...,"개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적..."
3,헌법,개성공업지구 지원에 관한 법률,The Government may grant or lend to a local en...,정부는 개성공업지구 현지기업에 「남북협력기금법」 에 따른 남북협력기금을 지원 또는 ...
4,헌법,개성공업지구 지원에 관한 법률,In addition to the business support under this...,이 법에 정한 기업 지원에 관한 사항 이외에 다른 법률이 정하고 있는 인력ㆍ기술개발...
...,...,...,...,...
276353,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(1) A Korean national worker for the United St...,① 국방부장관이 제5조에 따라 결정한 사항에 대하여 이의가 있는 주한미군 소속 한국...
276354,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"(2) The Minister of National Defense shall, wi...",② 국방부장관은 이의신청을 받은 날부터 7일 이내에 그 이의신청에 대하여 결정하고 ...
276355,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,(3) Procedures for appeal and giving notice of...,③ 제1항과 제2항에 따른 신청 절차와 결정 통지 등에 필요한 사항은 국방부령으로 ...
276356,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"(1) Costs necessary to pay subsidies, etc. und...",① 이 법에 따른 지원금의 지급 등에 필요한 비용은 국가에서 부담한다.


In [80]:
# 빈 문자열이나 Nan인 경우 확인

import numpy as np
df[df.en==np.NaN]

,category,law_name,en,ko


In [82]:
df[df.ko==np.NaN]

,category,law_name,en,ko


In [81]:
df[df.en=='nan']

,category,law_name,en,ko


In [ ]:
df[df.ko=='nan']

,category,law_name,en,ko


In [ ]:
df[df.en=='']

,category,law_name,en,ko


In [ ]:
df[df.ko=='']

,category,law_name,en,ko


In [ ]:
# 불용어 처리 및 앞뒤 공백 제거

df.iloc[:100, 2]

0     The purpose of this Act is to enhance inter-Ko...
1     The terms used in this Act are defined as foll...
2     This Act shall prevail over other statutes wit...
3     The Government may grant or lend to a local en...
4     In addition to the business support under this...
                            ...                        
95    (2) Before conducting a fact-finding survey, t...
96    (3) A fact-finding survey shall be conducted b...
97    (4) If necessary for efficient fact-finding su...
98    (5) Upon conducting a fact-finding survey, the...
99    (1) For purposes of applying the subparagraphs...
Name: en, Length: 100, dtype: object

In [25]:
import re
df.iloc[:100].en.apply(lambda x: re.sub('\'|"|\([^\)]*\)|「|」|<[^>]*>|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮',
                                        '', str(x).strip()))

0     The purpose of this Act is to enhance inter-Ko...
1     The terms used in this Act are defined as foll...
2     This Act shall prevail over other statutes wit...
3     The Government may grant or lend to a local en...
4     In addition to the business support under this...
                            ...                        
95    Before conducting a fact-finding survey, the M...
96    A fact-finding survey shall be conducted by me...
97    If necessary for efficient fact-finding survey...
98    Upon conducting a fact-finding survey, the Min...
99    For purposes of applying the subparagraphs of ...
Name: en, Length: 100, dtype: object

In [ ]:
df.iloc[:100, 3]

0     이 법은 개성공업지구의 개발ㆍ운영의 지원 및 개성공업지구에 투자하거나 출입ㆍ체류하는...
1     이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2009. 1. 30., 2...
2     개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적...
3     정부는 개성공업지구 현지기업에 「남북협력기금법」 에 따른 남북협력기금을 지원 또는 ...
4     이 법에 정한 기업 지원에 관한 사항 이외에 다른 법률이 정하고 있는 인력ㆍ기술개발...
                            ...                        
95    ② 통일부장관은 실태조사를 하려면 조사의 목적, 기간, 항목 및 내용과 조사 결과의...
96                       ③ 실태조사는 서면조사와 현장조사 등의 방법으로 한다.
97    ④ 통일부장관은 실태조사를 효율적으로 하기 위하여 필요한 경우에는 회계 등에 관한 ...
98                ⑤ 통일부장관은 실태조사를 한 경우에는 그 결과를 공표할 수 있다.
99    ① 법 제13조제1항 각 호 및 법 제15조제1항제4호의 적용을 위하여 국민연금관리...
Name: ko, Length: 100, dtype: object

In [27]:
import re
df.iloc[:100].ko.apply(lambda x: re.sub('\'|"|\([^\)]*\)|「|」|<[^>]*>|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮',
                                        '', str(x).strip())).apply(lambda x: re.sub('ㆍ', ' 및 ', str(x)))

0     이 법은 개성공업지구의 개발 및 운영의 지원 및 개성공업지구에 투자하거나 출입 및 ...
1                           이 법에서 사용하는 용어의 정의는 다음과 같다. 
2     개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적...
3     정부는 개성공업지구 현지기업에 남북협력기금법 에 따른 남북협력기금을 지원 또는 융자...
4     이 법에 정한 기업 지원에 관한 사항 이외에 다른 법률이 정하고 있는 인력 및 기술...
                            ...                        
95    통일부장관은 실태조사를 하려면 조사의 목적, 기간, 항목 및 내용과 조사 결과의 공...
96                         실태조사는 서면조사와 현장조사 등의 방법으로 한다.
97    통일부장관은 실태조사를 효율적으로 하기 위하여 필요한 경우에는 회계 등에 관한 전문...
98                  통일부장관은 실태조사를 한 경우에는 그 결과를 공표할 수 있다.
99    법 제13조제1항 각 호 및 법 제15조제1항제4호의 적용을 위하여 국민연금관리공단...
Name: ko, Length: 100, dtype: object

In [ ]:
df.iloc[0, 2]

'The purpose of this Act is to enhance inter-Korean exchanges and cooperation and further to contribute to the balanced development of the economy of the Korean nation by providing matters necessary for supporting the development and operation of the Gaeseong Industrial Complex and protecting and supporting South Korean residents (including corporations; hereinafter the same shall apply) who invests in, enter, exit, or stay in the Gaeseong Industrial Complex.'

In [8]:
df.iloc[0, 3]

'이 법은 개성공업지구의 개발ㆍ운영의 지원 및 개성공업지구에 투자하거나 출입ㆍ체류하는 남한주민(법인을 포함한다. 이하 같다)의 보호ㆍ지원에 관하여 필요한 사항을 규정함으로써 남북교류ㆍ협력을 증진하고 민족경제의 균형적인 발전에 기여함을 목적으로 한다.'

In [420]:
# 실제 데이터에 불용어 처리 적용

df.en = df.en.apply(lambda x: re.sub('\'|\"|\([^\)]*|\)|\[[^\]]*|\]|「|」|\<[^\>]*|\>|\{[^\)]*\}|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮|\*|○|⇒|ㅇ',
                                     '', str(x).strip().lstrip('-|?')))

In [421]:
df.ko = df.ko.apply(lambda x: re.sub('\'|\"|\([^\)]*|\)|\[[^\]]*|\]|「|」|\<[^\>]*|\>|\{[^\)]*\}|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮|\*|○|⇒|ㅇ',
                                     '', str(x).strip().lstrip('-|?'))).apply(lambda x: re.sub('ㆍ', ' 및 ', str(x)))

In [422]:
df

,category,law_name,en,ko
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발 및 운영의 지원 및 개성공업지구에 투자하거나 출입 및 ...
1,헌법,개성공업지구 지원에 관한 법률,The terms used in this Act are defined as foll...,이 법에서 사용하는 용어의 정의는 다음과 같다.
2,헌법,개성공업지구 지원에 관한 법률,This Act shall prevail over other statutes wit...,"개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적..."
3,헌법,개성공업지구 지원에 관한 법률,The Government may grant or lend to a local en...,정부는 개성공업지구 현지기업에 남북협력기금법 에 따른 남북협력기금을 지원 또는 융자...
4,헌법,개성공업지구 지원에 관한 법률,In addition to the business support under this...,이 법에 정한 기업 지원에 관한 사항 이외에 다른 법률이 정하고 있는 인력 및 기술...
...,...,...,...,...
276353,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,A Korean national worker for the United State...,국방부장관이 제5조에 따라 결정한 사항에 대하여 이의가 있는 주한미군 소속 한국인...
276354,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"The Minister of National Defense shall, withi...",국방부장관은 이의신청을 받은 날부터 7일 이내에 그 이의신청에 대하여 결정하고 결...
276355,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,Procedures for appeal and giving notice of de...,제1항과 제2항에 따른 신청 절차와 결정 통지 등에 필요한 사항은 국방부령으로 정한다.
276356,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"Costs necessary to pay subsidies, etc. under ...",이 법에 따른 지원금의 지급 등에 필요한 비용은 국가에서 부담한다.


In [132]:
# 크롤링 오류 찾기

df.loc[df.en.str.contains('Omitted')]

,category,law_name,en,ko
1417,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률 시행령,Omitted.,생략
3027,국회,국회사무처법,Omitted.,생략
3030,국회,국회사무처법,Omitted.,생략
4870,선거ㆍ정당,공직선거법,Omitted.,생략
4877,선거ㆍ정당,공직선거법,Omitted.,생략
...,...,...,...,...
273195,정보통신,정보통신기반 보호법 시행령,Omitted.,생략
273833,정보통신,정보통신산업 진흥법,Omitted.,생략
274828,정보통신,통신비밀보호법,Omitted.,생략
275742,외무,외무공무원법,Omitted.,생략


In [134]:
df.loc[df.en.str.contains('Omitted'), ['en']].value_counts()

en                
 Omitted.             376
 through  Omitted.     58
 and  Omitted.         41
 Omitted                4
  Omitted.              3
dtype: int64

In [135]:
df.loc[df.en.str.contains('Omitted'), ['ko']].value_counts()

ko                                                                                                              
 생략                                                                                                                 349
 및  생략                                                                                                               25
 내지 생략                                                                                                               22
 부터 까지 생략                                                                                                            19
 및 생략                                                                                                                17
생략                                                                                                                    5
 부터  까지 생략                                                                                                            2
 교육세법중 다음과 같이 개정한다.                            

In [57]:
df[df.ko=='청소년보호법시행령중 다음과 같이 개정한다.']

,category,law_name,en,ko
91899,문화ㆍ공보,출판문화산업 진흥법 시행령,Omitted.,청소년보호법시행령중 다음과 같이 개정한다.


In [75]:
df.iloc[91895:91905, 2:]

,en,ko
91895,In addition to the matters provided for in par...,제1항부터 제4항까지에서 규정한 사항 외에 포상금의 지급에 필요한 사항은 문화체육관...
91896,"The Minister of Culture, Sports and Tourism sh...",문화체육관광부장관은 법 제26조에 따라 다음 각 호에 따른 과태료의 부과·징수에 관...
91897,"The Minister of Culture, Sports and Tourism sh...",문화체육관광부장관은 법 제26조에 따라 법 제25조의2에 따른 포상금의 지급 업무를...
91898,The Enforcement Decree of the Registration of ...,출판사및인쇄소의등록에관한법률시행령 및 외국간행물수입배포에관한법률시행령은 이를 각각 ...
91899,Omitted.,청소년보호법시행령중 다음과 같이 개정한다.
91900,When applying the criteria for imposing admini...,이 영 시행 전의 위반행위에 대하여 과태료의 부과기준을 적용할 때에는 별표 2의 개...
91901,A disposition of imposition of an administrati...,이 영 시행 전의 위반행위로 받은 과태료 부과처분은 별표 2의 개정규정에 따른 위반...
91902,The purpose of this Act is to ensure the trans...,이 법은 우리 민족의 문화적 자산인 풍납토성의 보존 및 관리에 관하여 필요한 사항을...
91903,The terms used in this Act are defined as foll...,이 법에서 사용하는 용어의 뜻은 다음과 같다.
91904,The State and local governments shall cooperat...,국가와 지방자치단체는 보존 및 관리사업을 원활하고 효율적으로 추진하기 위하여 상호 ...


In [160]:
df.loc[df.en.str.contains('Deleted')]

,category,law_name,en,ko
489,헌법,남북교류협력에 관한 법률,Deleted.,삭제
1083,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률,Deleted.,삭제
1247,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률 시행령,Deleted.,삭제
1285,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률 시행령,Deleted.,삭제
1310,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률 시행령,Deleted.,삭제
...,...,...,...,...
276265,외무,한국국제협력단법,Deleted.,삭제
276266,외무,한국국제협력단법,Deleted.,삭제
276268,외무,한국국제협력단법,Deleted.,삭제
276309,외무,한국국제협력단법,Deleted.,삭제


In [161]:
df.loc[df.en.str.contains('Deleted'), ['en']].value_counts()

en                    
Deleted.                  2411
Deleted;                     7
Deleted                      3
Deleted. >                   1
Deleted. Jan. 6, 2015>       1
Deleted:                     1
and  Deleted.                1
dtype: int64

In [162]:
df.loc[df.en.str.contains('Deleted'), ['ko']].value_counts()

ko                                                                                                                                                    
삭제                                                                                                                                                        2412
                                                                                                                                                             7
생략                                                                                                                                                           3
영 제93조의4제11항에서 “기획재정부령으로 정하는 주식”이란 자본시장과 금융투자업에 관한 법률 제9조제15항제3호에 따른 주권상장법인의 주식을 말한다.                                                                        1
영 제93조의4제9항에서 “기획재정부령으로 정하는 주식”이란 자본시장과 금융투자업에 관한 법률 제9조제15항제3호에 따른 주권상장법인의 주식을 말한다. 다만, 소득세법 시행령 제157조제4항 각 호의 어느 하나에 해당하는 주권상장법인의 대주주가 보유한 주식은 제외한다.       1
제1항에 따른 주식에서 발생한 양도차손을 계산할 때 같은 종목의 주식을 2회 이상 취득한 경우 그

In [163]:
df.loc[df.ko=='영 제93조의4제11항에서 “기획재정부령으로 정하는 주식”이란 자본시장과 금융투자업에 관한 법률 제9조제15항제3호에 따른 주권상장법인의 주식을 말한다.']

,category,law_name,en,ko
121088,내국세,조세특례제한법 시행규칙,Deleted.,영 제93조의4제11항에서 “기획재정부령으로 정하는 주식”이란 자본시장과 금융투자업...


In [63]:
df.iloc[121084:121100]

,category,law_name,en,ko
121084,내국세,조세특례제한법 시행규칙,The certificate of deposit in a housing saving...,제1항에 따른 주택마련저축납입증명서는 연도말 현재의 납입액이 표시되어 있는 주택마련...
121085,내국세,조세특례제한법 시행규칙,“A document certifying the period of military ...,영 제81조제15항제2호에서 “기획재정부령으로 정하는 병역복무기간을 증명하는 서류”...
121086,내국세,조세특례제한법 시행규칙,Deleted.,영 제93조의4제9항에서 “기획재정부령으로 정하는 주식”이란 자본시장과 금융투자업에...
121087,내국세,조세특례제한법 시행규칙,Deleted.,제1항에 따른 주식에서 발생한 양도차손을 계산할 때 같은 종목의 주식을 2회 이상 ...
121088,내국세,조세특례제한법 시행규칙,Deleted.,영 제93조의4제11항에서 “기획재정부령으로 정하는 주식”이란 자본시장과 금융투자업...
121089,내국세,조세특례제한법 시행규칙,Only where a trust business entity is unable t...,신탁업자는 개인종합자산관리계좌를 법 제91조의18제3항제3호 각 목의 재산으로 운용...
121090,내국세,조세특례제한법 시행규칙,Interest income accrued from the management un...,제4항에 따른 운용을 통해 발생한 이자소득은 법 제91조의18제5항에 따른 이자소득...
121091,내국세,조세특례제한법 시행규칙,“Written form” under Article 99-6 of the Decr...,영 제99조의6제3항에 따른 문서는 체납액 납부계획서를 말한다.
121092,내국세,조세특례제한법 시행규칙,“Application” under Article 99-6 of the Decre...,"영 제99조의6제8항에 따른 신청서는 납부고지 유예, 지정납부기한 및 독촉장에서 정..."
121093,내국세,조세특례제한법 시행규칙,Deleted.,삭제


In [423]:
# 'Ommited', 'Deleted' 등 영어와 한글 매칭이 되지 않은 부분은 삭제
# 이는 크롤링 과정에서 발생한 오류가 아니라, 사이트 내부 소스코드 상의 오류이므로 생략해도 무방(순서 밀림 없음)

omitted_id = df.loc[df.en.str.contains('Omitted')].index
deleted_id = df.loc[df.en.str.contains('Deleted')].index

In [424]:
omitted_id

Int64Index([  1417,   3027,   3030,   4870,   4877,   4884,   6257,   7568,
              8791,   9164,
            ...
            270130, 270231, 271420, 271819, 272872, 273195, 273833, 274828,
            275742, 276307],
           dtype='int64', length=482)

In [425]:
deleted_id

Int64Index([   489,   1083,   1247,   1285,   1310,   1412,   1420,   2065,
              2188,   2528,
            ...
            276015, 276016, 276058, 276221, 276264, 276265, 276266, 276268,
            276309, 276324],
           dtype='int64', length=2425)

In [426]:
df.drop(index=omitted_id, inplace=True)
df.drop(index=deleted_id, inplace=True)

In [427]:
df.loc[df.en.str.contains('Omitted')]

,category,law_name,en,ko


In [428]:
df.loc[df.en.str.contains('Deleted')]

,category,law_name,en,ko


In [429]:
# 수식은 전부 삭제(문장이 아니므로)

formula_id = df.loc[df.en.str.contains('=|\+|÷')].index
formula_id

Int64Index([  3488,   3495,   9169,  48314,  48347,  49426,  49427,  49429,
             49430,  51221,  69127,  81842, 109834, 109844, 109846, 112917,
            113862, 113870, 113871, 113872, 114890, 116499, 117487, 117488,
            117489, 119125, 119126, 119139, 119140, 121981, 122397, 149642,
            164408, 164410, 164412, 164414, 164419, 164422, 164429, 164431,
            164437, 164439, 164443, 164447, 164458, 164461, 164466, 164514,
            180363, 182244, 183014, 194841, 202344, 202347, 202348, 202514,
            202515, 230552, 237540, 238769, 238770, 238771, 240333, 242744,
            243111, 243112, 243144, 243150, 244301, 254813],
           dtype='int64')

In [430]:
df.drop(index=formula_id, inplace=True)
df

,category,law_name,en,ko
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발 및 운영의 지원 및 개성공업지구에 투자하거나 출입 및 ...
1,헌법,개성공업지구 지원에 관한 법률,The terms used in this Act are defined as foll...,이 법에서 사용하는 용어의 정의는 다음과 같다.
2,헌법,개성공업지구 지원에 관한 법률,This Act shall prevail over other statutes wit...,"개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적..."
3,헌법,개성공업지구 지원에 관한 법률,The Government may grant or lend to a local en...,정부는 개성공업지구 현지기업에 남북협력기금법 에 따른 남북협력기금을 지원 또는 융자...
4,헌법,개성공업지구 지원에 관한 법률,In addition to the business support under this...,이 법에 정한 기업 지원에 관한 사항 이외에 다른 법률이 정하고 있는 인력 및 기술...
...,...,...,...,...
276353,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,A Korean national worker for the United State...,국방부장관이 제5조에 따라 결정한 사항에 대하여 이의가 있는 주한미군 소속 한국인...
276354,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"The Minister of National Defense shall, withi...",국방부장관은 이의신청을 받은 날부터 7일 이내에 그 이의신청에 대하여 결정하고 결...
276355,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,Procedures for appeal and giving notice of de...,제1항과 제2항에 따른 신청 절차와 결정 통지 등에 필요한 사항은 국방부령으로 정한다.
276356,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"Costs necessary to pay subsidies, etc. under ...",이 법에 따른 지원금의 지급 등에 필요한 비용은 국가에서 부담한다.


In [431]:
df.loc[df.en.str.contains('=|\+|÷')]

,category,law_name,en,ko


In [432]:
formula_id2 = df.loc[df.ko.str.contains('=|\+|÷')].index
formula_id2

Int64Index([109833, 109845, 115872, 116508, 163215, 163216, 163833], dtype='int64')

In [433]:
df.drop(index=formula_id2, inplace=True)
df

,category,law_name,en,ko
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발 및 운영의 지원 및 개성공업지구에 투자하거나 출입 및 ...
1,헌법,개성공업지구 지원에 관한 법률,The terms used in this Act are defined as foll...,이 법에서 사용하는 용어의 정의는 다음과 같다.
2,헌법,개성공업지구 지원에 관한 법률,This Act shall prevail over other statutes wit...,"개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적..."
3,헌법,개성공업지구 지원에 관한 법률,The Government may grant or lend to a local en...,정부는 개성공업지구 현지기업에 남북협력기금법 에 따른 남북협력기금을 지원 또는 융자...
4,헌법,개성공업지구 지원에 관한 법률,In addition to the business support under this...,이 법에 정한 기업 지원에 관한 사항 이외에 다른 법률이 정하고 있는 인력 및 기술...
...,...,...,...,...
276353,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,A Korean national worker for the United State...,국방부장관이 제5조에 따라 결정한 사항에 대하여 이의가 있는 주한미군 소속 한국인...
276354,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"The Minister of National Defense shall, withi...",국방부장관은 이의신청을 받은 날부터 7일 이내에 그 이의신청에 대하여 결정하고 결...
276355,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,Procedures for appeal and giving notice of de...,제1항과 제2항에 따른 신청 절차와 결정 통지 등에 필요한 사항은 국방부령으로 정한다.
276356,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"Costs necessary to pay subsidies, etc. under ...",이 법에 따른 지원금의 지급 등에 필요한 비용은 국가에서 부담한다.


In [434]:
df.loc[df.ko.str.contains('=|\+|÷')]

,category,law_name,en,ko


In [412]:
# % -> 영어에서는 'percent', 국어에서는 '퍼센트'로 통일

df.loc[df.en.str.contains('%')]

,category,law_name,en,ko
1321,헌법,북한이탈주민의 보호 및 정착지원에 관한 법률 시행령,Ratio ... prescribed by Presidential Decree in...,법 제17조의5제1항제2호에서 “대통령령으로 정하는 비율”이란 연간 5퍼센트를 말한다.
4417,선거ㆍ정당,공직선거법,The percentage of votes obtained in the electi...,제2항의 비례대표국회의원선거 득표비율은 각 의석할당정당의 득표수를 모든 의석할당정당...
10038,행정일반,책임운영기관의 설치·운영에 관한 법률,Affiliated responsible administrative agencies...,제1항에 따라 책임운영기관특별회계로 운영할 필요가 있는 소속책임운영기관은 재정수입 ...
10170,행정일반,청년기본법,The head of a relevant central administrative ...,관계 중앙행정기관의 장 및 시 및 도지사는 청년정책을 주로 다루는 위원회를 구성할 ...
11562,국가공무원,공공재정 부정청구 금지 및 부정이익 환수 등에 관한 법률 시행령,A monetary reward under Article 23 of the Act...,법 제23조제3항에 따른 보상금은 보상대상가액의 30퍼센트로 한다.
...,...,...,...,...
271833,정보통신,전파법 시행령,Where the security deposit and price for radio...,법 제11조제5항 및 제6항에 따라 보증금 및 주파수할당 대가를 방송통신발전기금 및...
272172,정보통신,전파법 시행령,Where an inspection is conducted on each trans...,법 제10조에 따라 과학기술정보통신부장관이 할당한 주파수를 이용하는 기간통신사업자의...
272173,정보통신,전파법 시행령,In cases of the following radio stations in wh...,제69조제1항제1호 및 제2호에 해당하는 무선설비를 둘 이상의 기간통신사업자나 방송...
272267,정보통신,정보보호산업의 진흥에 관한 법률,Where a person who has disclosed the current s...,제1항에 따라 정보보호 현황을 공개한 자가 정보통신망 이용촉진 및 정보보호 등에 관...


In [435]:
df.loc[df.ko.str.contains('%')]

,category,law_name,en,ko
95726,과학ㆍ기술,산업기술혁신사업 공통 운영요령,A participating researcher for a project for ...,제1항에 따라 사업을 신청하는 해당 사업의 참여연구원은 참여율이 10% 이상이어야...
95758,과학ㆍ기술,산업기술혁신사업 공통 운영요령,Where a middle-standng or small-medium enterp...,중견·중소기업이 외부 기술도입을 한 경우 평가위원회 또는 연구발표회의 심의를 거쳐...
95813,과학ㆍ기술,산업기술혁신사업 공통 운영요령,The head of an agency vested with exclusive r...,전담기관의 장은 중단 평가를 받은 과제에 한하여 원인보고서 작성·논문작성·특허출원...
100619,재정ㆍ경제일반,경품류제공에관한불공정거래행위의유형및기준지정고시,If the total value of premiums offered by a b...,사업자가 소비자현상경품으로 제공하는 경품가액의 합계액이 경품부상품 또는 용역의 예...
100620,재정ㆍ경제일반,경품류제공에관한불공정거래행위의유형및기준지정고시,"Notwithstanding paragraph , the upper limit f...","제1항의 규정에 불구하고 정기간행물출판업, 경기후원업 및 방송업을 영위하는 사업자..."
100621,재정ㆍ경제일반,경품류제공에관한불공정거래행위의유형및기준지정고시,The expected sales price referred to in parag...,"제1항과 제2항의 예상매출액은 다음 각호의 방법에 의하여 산정한다. 다만, 매출액..."
108194,재정ㆍ경제일반,외국인투자지역운영지침,A down payment for the purchase of land in a ...,"외국인투자지역 부지매입시 계약금은 국가와 지자체가 총액의 50%씩 각각 분담하며,..."
108241,재정ㆍ경제일반,외국인투자지역운영지침,The area allowed for a subcontractor to move ...,협력업체의 입주허용 면적은 해당 입주업체의 공장건축면적의 30%를 초과 할 수 ...
108252,재정ㆍ경제일반,외국인투자지역운영지침,The area that an entity engaged in electricit...,제1항에 따라 입주가 허용된 발전사업자의 입주허용 면적은 공장건축 면적의 30%를...
108293,재정ㆍ경제일반,외국인투자지역운영지침,A newly or additionally designated zone or b...,"신규 및 추가 지정 지역 또는 건물은 즉시 입주가 가능하여야 하며, 외국인투자기업..."


In [436]:
df.en = df.en.apply(lambda x: re.sub('%', 'percent', str(x)))
df.ko = df.ko.apply(lambda x: re.sub('%', '퍼센트', str(x)))

In [437]:
df.loc[df.en.str.contains('%')]

,category,law_name,en,ko


In [438]:
df.loc[df.ko.str.contains('%')]

,category,law_name,en,ko


In [439]:
# 마지막으로 인덱스 초기화

df.reset_index(drop=True, inplace=True)

In [440]:
df

,category,law_name,en,ko
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발 및 운영의 지원 및 개성공업지구에 투자하거나 출입 및 ...
1,헌법,개성공업지구 지원에 관한 법률,The terms used in this Act are defined as foll...,이 법에서 사용하는 용어의 정의는 다음과 같다.
2,헌법,개성공업지구 지원에 관한 법률,This Act shall prevail over other statutes wit...,"개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적..."
3,헌법,개성공업지구 지원에 관한 법률,The Government may grant or lend to a local en...,정부는 개성공업지구 현지기업에 남북협력기금법 에 따른 남북협력기금을 지원 또는 융자...
4,헌법,개성공업지구 지원에 관한 법률,In addition to the business support under this...,이 법에 정한 기업 지원에 관한 사항 이외에 다른 법률이 정하고 있는 인력 및 기술...
...,...,...,...,...
273369,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,A Korean national worker for the United State...,국방부장관이 제5조에 따라 결정한 사항에 대하여 이의가 있는 주한미군 소속 한국인...
273370,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"The Minister of National Defense shall, withi...",국방부장관은 이의신청을 받은 날부터 7일 이내에 그 이의신청에 대하여 결정하고 결...
273371,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,Procedures for appeal and giving notice of de...,제1항과 제2항에 따른 신청 절차와 결정 통지 등에 필요한 사항은 국방부령으로 정한다.
273372,-,주한미군 소속 한국인 근로자의 지원을 위한 특별법,"Costs necessary to pay subsidies, etc. under ...",이 법에 따른 지원금의 지급 등에 필요한 비용은 국가에서 부담한다.


In [441]:
# 전처리 완료한 데이터 따로 저장

df.to_csv('/content/drive/MyDrive/KBdata/processed_df.csv', index=False)

# 4. 토크나이징
- 한국어 형태소 분석기 활용해서 품사태깅
  
  - Hannanum (고유명사 가장 분류 잘 하는 듯)
  - Okt
  - Kkma

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 33.9 MB/s eta 0:00:00


In [ ]:
df.iloc[:3, -1]

0    이 법은 개성공업지구의 개발ㆍ운영의 지원 및 개성공업지구에 투자하거나 출입ㆍ체류하는...
1                          이 법에서 사용하는 용어의 정의는 다음과 같다. 
2    개성공업지구에 대한 지원, 왕래와 교역에 관하여는 다른 법률에 우선하여 이 법을 적...
Name: ko, dtype: object

In [ ]:
from konlpy.tag import Hannanum

hannanum = Hannanum()
kor = df.iloc[:3, -1]

for k in kor:
  morphs = hannanum.morphs(k)
  pos = hannanum.pos(k)
  print('morphs:',morphs)
  print('pos:',pos)

morphs: ['이', '법', '은', '개성공업지구', '의', '개발ㆍ운영', '의', '지원', '및', '개성공업지구', '에', '투자하', '이', '거나', '출입ㆍ체류하', '는', '남한주민', '의', '보호ㆍ지원', '에', '관하', '어', '필요', '하', 'ㄴ', '사항', '을', '규정함', '으로써', '남북교류ㆍ협력', '을', '증진', '하고', '민족경제', '의', '균형적', '이', 'ㄴ', '발전', '에', '기이', '어', '하', 'ㅁ', '을', '목적', '으로', '하', 'ㄴ다', '.']
pos: [('이', 'M'), ('법', 'N'), ('은', 'J'), ('개성공업지구', 'N'), ('의', 'J'), ('개발ㆍ운영', 'N'), ('의', 'J'), ('지원', 'N'), ('및', 'M'), ('개성공업지구', 'N'), ('에', 'J'), ('투자하', 'N'), ('이', 'J'), ('거나', 'E'), ('출입ㆍ체류하', 'N'), ('는', 'J'), ('남한주민', 'N'), ('의', 'J'), ('보호ㆍ지원', 'N'), ('에', 'J'), ('관하', 'P'), ('어', 'E'), ('필요', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('사항', 'N'), ('을', 'J'), ('규정함', 'N'), ('으로써', 'J'), ('남북교류ㆍ협력', 'N'), ('을', 'J'), ('증진', 'N'), ('하고', 'J'), ('민족경제', 'N'), ('의', 'J'), ('균형적', 'N'), ('이', 'J'), ('ㄴ', 'E'), ('발전', 'N'), ('에', 'J'), ('기이', 'P'), ('어', 'E'), ('하', 'P'), ('ㅁ', 'E'), ('을', 'J'), ('목적', 'N'), ('으로', 'J'), ('하', 'P'), ('ㄴ다', 'E'), ('.', 'S')]
morphs: ['이', '법', '에서', 

In [ ]:
from konlpy.tag import Okt

okt = Okt()
kor = df.iloc[:3, -1]

for k in kor:
  morphs = okt.morphs(k)
  pos = okt.pos(k)
  print('morphs:',morphs)
  print('pos:',pos)

morphs: ['이', '법', '은', '개성', '공업', '지구', '의', '개발', 'ㆍ', '운영', '의', '지원', '및', '개성', '공업', '지구', '에', '투자', '하', '거나', '출입', 'ㆍ', '체류', '하는', '남한', '주민', '의', '보호', 'ㆍ', '지원', '에', '관', '하여', '필요한', '사항', '을', '규정', '함', '으로써', '남북', '교류', 'ㆍ', '협력', '을', '증진', '하고', '민족', '경제', '의', '균형', '적', '인', '발전', '에', '기여', '함', '을', '목적', '으로', '한다', '.']
pos: [('이', 'Noun'), ('법', 'Noun'), ('은', 'Josa'), ('개성', 'Noun'), ('공업', 'Noun'), ('지구', 'Noun'), ('의', 'Josa'), ('개발', 'Noun'), ('ㆍ', 'Foreign'), ('운영', 'Noun'), ('의', 'Josa'), ('지원', 'Noun'), ('및', 'Noun'), ('개성', 'Noun'), ('공업', 'Noun'), ('지구', 'Noun'), ('에', 'Josa'), ('투자', 'Noun'), ('하', 'Suffix'), ('거나', 'Noun'), ('출입', 'Noun'), ('ㆍ', 'Foreign'), ('체류', 'Noun'), ('하는', 'Verb'), ('남한', 'Noun'), ('주민', 'Noun'), ('의', 'Josa'), ('보호', 'Noun'), ('ㆍ', 'Foreign'), ('지원', 'Noun'), ('에', 'Josa'), ('관', 'Noun'), ('하여', 'Verb'), ('필요한', 'Adjective'), ('사항', 'Noun'), ('을', 'Josa'), ('규정', 'Noun'), ('함', 'Noun'), ('으로써', 'Josa'), ('남북', 'Noun'), 

In [ ]:
from konlpy.tag import Kkma

kkma = Kkma()
kor = df.iloc[:3, -1]

for k in kor:
  morphs = kkma.morphs(k)
  pos = kkma.pos(k)
  print('morphs:',morphs)
  print('pos:',pos)

morphs: ['이', '법', '은', '개성', '공업', '지구의', '개발ㆍ운영', '의', '지원', '및', '개성', '공업', '지구', '에', '투자', '하', '거나', '출입ㆍ체류하', '는', '남한', '주민', '의', '보호ㆍ지원', '에', '관하', '여', '필요', '하', 'ㄴ', '사항', '을', '규정', '하', 'ㅁ', '으로써', '남북교류ㆍ협력', '을', '증진', '하', '고', '민족', '경제', '의', '균형', '적', '이', 'ㄴ', '발전', '에', '기여', '하', 'ㅁ', '을', '목적', '으로', '하', 'ㄴ다', '.']
pos: [('이', 'MDT'), ('법', 'NNG'), ('은', 'JX'), ('개성', 'NNG'), ('공업', 'NNG'), ('지구의', 'NNG'), ('개발ㆍ운영', 'UN'), ('의', 'JKG'), ('지원', 'NNG'), ('및', 'MAG'), ('개성', 'NNG'), ('공업', 'NNG'), ('지구', 'NNG'), ('에', 'JKM'), ('투자', 'NNG'), ('하', 'XSV'), ('거나', 'ECE'), ('출입ㆍ체류하', 'UN'), ('는', 'JX'), ('남한', 'NNG'), ('주민', 'NNG'), ('의', 'JKG'), ('보호ㆍ지원', 'UN'), ('에', 'JKM'), ('관하', 'VV'), ('여', 'ECS'), ('필요', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('사항', 'NNG'), ('을', 'JKO'), ('규정', 'NNG'), ('하', 'XSV'), ('ㅁ', 'ETN'), ('으로써', 'JKM'), ('남북교류ㆍ협력', 'UN'), ('을', 'JKO'), ('증진', 'NNG'), ('하', 'XSV'), ('고', 'ECE'), ('민족', 'NNG'), ('경제', 'NNG'), ('의', 'JKG'), ('균형', 'NNG'),